## IC/A1 data per file: comparing datasets
#### DS December 2018

In this notebook loads the csv files of each batch of data from Daniela's AC and IC tone in noise and tone vs noise data, Sean's IC tone vs noise ferret data, as well asd Stephen's A1 old data reanalized in Slee and David 2015. These data is then merged to information about STRF snr, target frequency, and more tuning info. The goal is to compare MI, gain, and offeset across these data sets.

Batches:
* **d_AC_307** DS, SU and MU, onBF and offBF
* **d_AC_312** SVD, SU and MU, offBF
* **d_AC_311** SVD, SU and MU, onBF
* **d_IC_313** DS, SU and MU, onBF and offBF, pup and nopup
* **d_IC_295** SJS, SU, onBF and offBF

In [ ]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline
os.getcwd()

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 60)

In [ ]:
# AC SU+MU 307 DS' data (onBF and offBF together)
# run using beh_only_plot function in mod_per_state.py with state_list = ['st.fil0', 'st.fil'], 
# basemodel = '-ref-psthfr.s_stategain.S'
# loader = 'psth.fs20-ld-'
# fitter = '_jk.nf20-basic'
# get_model_results_per_state_model ---> returning df and saved on /auto/users/daniela/code/python_ACIC_proj/d_fil_307.csv 
# 12/18/18 DS
d_AC_307 = pd.read_csv('d_fil_307.csv')
d_AC_307 = d_AC_307.drop(['Unnamed: 0'], axis=1)
d_AC_307['experimenter'] = 'DS'
d_AC_307['area'] = 'AC'
d_AC_307['onBF'] = 'TBD'
d_AC_307['SU'] = False
d_AC_307.loc[d_AC_307['isolation']>=90.0, 'SU'] = True
d_AC_307['animal'] = d_AC_307['cellid'].map(lambda x: x[:3])
d_AC_307['task'] = 'TIN'

In [ ]:
d_AC_307.head()

In [ ]:
# AC 311 SVD's data (onBF only)
# run using beh_only_plot function in mod_per_state.py with state_list = ['st.fil0', 'st.fil'], 
# basemodel = '-ref-psthfr.s_stategain.S'
# loader = 'psth.fs20-ld-'
# fitter = '_jk.nf20-basic'
# get_model_results_per_state_model ---> returning df and saved on /auto/users/daniela/code/python_ACIC_proj/d_fil_311.csv 
# 12/18/18 DS
d_AC_311 = pd.read_csv('d_fil_311.csv')
d_AC_311 = d_AC_311.drop(['Unnamed: 0'], axis=1)
d_AC_311['onBF'] = True
d_AC_311['experimenter'] = 'SVD'
d_AC_311['area'] = 'AC'
d_AC_311['SU'] = False
d_AC_311['animal'] = d_AC_311['cellid'].map(lambda x: x[:3])
d_AC_311['task'] = 'TvN'
d_AC_311.loc[d_AC_311['isolation']>=90.0, 'SU'] = True

In [ ]:
d_AC_311.head()

In [ ]:
# AC 312 SVD's data offBF only
d_AC_312 = pd.read_csv('d_fil_312.csv')
d_AC_312 = d_AC_312.drop(['Unnamed: 0'], axis=1)
d_AC_312['onBF'] = False
d_AC_312['experimenter'] = 'SVD'
d_AC_312['area'] = 'AC'
d_AC_312['SU'] = False
d_AC_312.loc[d_AC_312['isolation']>=90.0, 'SU'] = True
d_AC_312['animal'] = d_AC_312['cellid'].map(lambda x: x[:3])
d_AC_312['task'] = 'TvN'

In [ ]:
d_AC_312.head()

In [ ]:
# IC SU only 305 DS' data onBF+offBF (pupil+nopupil dataset)
d_IC_305 = pd.read_csv('d_fil_305.csv') # run with stategain model on October 31st
d_IC_305 = d_IC_305.drop(['Unnamed: 0'], axis=1)
d_IC_305['onBF'] = 'TBD'
d_IC_305['experimenter'] = 'DS'
d_IC_305['area'] = 'IC'
d_IC_305['animal'] = d_IC_305['cellid'].map(lambda x: x[:3])
d_IC_305['task'] = 'TIN'
d_IC_305.loc[d_IC_305['animal']=='ley', 'task'] = 'TvN'
d_IC_305['SU'] = True

In [ ]:
d_IC_305.head()

In [ ]:
# IC SU+MU 309 DS' data onBF+offBF (pupil only dataset)
d_IC_309 = pd.read_csv('d_fil_309.csv') # run with stategain model on October 31st
d_IC_309 = d_IC_309.drop(['Unnamed: 0'], axis=1)
d_IC_309['onBF'] = 'TBD'
d_IC_309['experimenter'] = 'DS'
d_IC_309['area'] = 'IC'
d_IC_309['animal'] = d_IC_309['cellid'].map(lambda x: x[:3])
d_IC_309['task'] = 'TIN'
d_IC_309.loc[d_IC_309['animal']=='ley', 'task'] = 'TvN'
d_IC_309['SU'] = False
d_IC_309.loc[d_IC_309['isolation']>=90.0, 'SU'] = True

In [ ]:
d_IC_309.head()

In [ ]:
# IC SU+MU 313 DS' data onBF+offBF (pupil+nopupil dataset)
d_IC_313 = pd.read_csv('d_fil_313.csv') # run with stategain model on December 31st
d_IC_313 = d_IC_313.drop(['Unnamed: 0'], axis=1)
d_IC_313['onBF'] = 'TBD'
d_IC_313['experimenter'] = 'DS'
d_IC_313['area'] = 'IC'
d_IC_313['animal'] = d_IC_313['cellid'].map(lambda x: x[:3])
d_IC_313['task'] = 'TIN'
d_IC_313.loc[d_IC_313['animal']=='ley', 'task'] = 'TvN'
d_IC_313['SU'] = False
d_IC_313.loc[d_IC_313['isolation']>=90.0, 'SU'] = True

In [ ]:
d_IC_313.tail()

In [ ]:
# IC SU 295 Sean's data 
d_IC_295 = pd.read_csv('d_295_IC_old.csv')
d_IC_295 = d_IC_295.drop(['Unnamed: 0'], axis=1)
d_IC_295['onBF'] = 'TBD'
d_IC_295['experimenter'] = 'SJS'
d_IC_295['animal'] = d_IC_295['cellid'].map(lambda x: x[:3])
d_IC_295['area'] = 'IC'
d_IC_295['task'] = 'TvN'
d_IC_295['SU'] = True

In [ ]:
d_IC_295.head()

In [ ]:
# concat dataframes 
# d_AC_307 DS, SU and MU, onBF and offBF
# d_AC_312 SVD, SU and MU, offBF
# d_AC_311 SVD, SU and MU, onBF
# d_IC_313 DS, SU and MU, onBF and offBF, pup and nopup
# d_IC_295 SJS, SU, onBF and offBF
d_all = pd.concat([d_AC_307, d_AC_312, d_AC_311, d_IC_313, d_IC_295], sort=False)

In [ ]:
# add columns with R2
d_all['R2'] = d_all['r']**2 * np.sign(d_all['r'])

In [ ]:
d_all.head()

In [ ]:
#d_all[d_all['cellid']=='BRT016f-a1']

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_295.csv')
# IC data Sean's SU only
d_295_strf = pd.read_csv('tuning_info_batch_295.csv')
d_295_strf = d_295_strf.drop(['Unnamed: 24'], axis=1)

In [ ]:
d_295_strf.head()

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_307.csv')
# AC data DS' SU+MU
d_307_strf = pd.read_csv('tuning_info_batch_307.csv')
d_307_strf = d_307_strf.drop(['Unnamed: 43'], axis=1)

In [ ]:
d_307_strf.head()

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_309.csv')
# IC data DS' SU (pupil+nopupil dataset)
d_305_strf = pd.read_csv('tuning_info_batch_305.csv')
d_305_strf = d_305_strf.drop(['Unnamed: 43'], axis=1)

In [ ]:
d_305_strf.head()

In [ ]:
# generated by running get_ptd_tuning in MATLAB and then running struct2csv(cellframe, 'tuning_info_batch_309.csv')
# IC data DS' SU+MU (pupil+nopupil dataset)
d_313_strf = pd.read_csv('tuning_info_batch_313.csv')
d_313_strf = d_313_strf.drop(['Unnamed: 48'], axis=1)

In [ ]:
d_313_strf.head()

In [ ]:
# concat all strf dataframes
d_strf = pd.concat([d_295_strf, d_307_strf, d_313_strf], sort=False)

In [ ]:
d_strf.head()

In [ ]:
# merge all into one giant df
df = pd.merge(left=d_all, right=d_strf, how='outer', on='cellid')

In [ ]:
# len(d_295_strf[d_295_strf['ACTIVE_0'].isna()])
# 64 NAN for ACTIVE_0
# 10 not nan for ACTIVE_0  (No pre-passive)

In [ ]:
df.columns

In [ ]:
def fix_TBD_onBF(df):
    '''This function takes the monster dataframe and adds True to the column 'onBF' when 'ACTIVE_1_tardist' is
    within 0.5 --> half an octave between unit BF and target frequency in ACTIVE 1'''
    BF_TBD = (df['onBF']=='TBD')
    df.loc[BF_TBD, 'onBF'] = df.loc[BF_TBD, 'ACTIVE_1_tardist'].map(lambda x: abs(x)<=0.5)
    return df

In [ ]:
# check before and after applying the func to make sure it works
df['onBF'].value_counts()

In [ ]:
df = fix_TBD_onBF(df)

In [ ]:
df['onBF'].value_counts()

In [ ]:
df.head()

## Comparisons of gain, offset and MI between datasets

In [ ]:
# variables to slice up df 
AC = df['area']=='AC'
IC = df['area']=='IC'
onBF = df['onBF']==True
offBF = df['onBF']==False

df_fil = df[df['state_sig']=='st.fil']
active1 = df_fil['state_chan']=='ACTIVE_1'
passive1 = df_fil['state_chan']=='PASSIVE_1'

In [ ]:
def plotme(df, x_col, y_col, title=None, savefigure=False):
    #ax = sns.violinplot(x=x_col, y=y_col, data=df)
    ax = plt.figure(num=None, figsize=(6,4), dpi=80, facecolor='w', edgecolor='k')
    ax = sns.boxplot(x=x_col, y=y_col, data=df)
    ax =sns.swarmplot(x=x_col, y=y_col, data=df, color='black')
    plt.axhline(color='k', linestyle='--', linewidth=0.5)
    plt.ylabel(y_col)
    #ax.set_aspect(aspect=0.5)
    if title:
        plt.title(title)
    if savefigure:
        plt.savefig(title+x_col+y_col+'.pdf')

In [ ]:
plotme(df[AC & active1], 'animal', 'g', title='AC_P0A1_gain_per_animal')

In [ ]:
plotme(df[AC & active1], 'task', 'g', title='AC_P0A1_gain_per_task')

In [ ]:
plotme(df[AC & active1], 'task', 'd', title='AC_P0A1_offset_per_task')

In [ ]:
plotme(df[AC & active1], 'task', 'MI', title='AC_P0A1_MI_per_task')

In [ ]:
plotme(df[AC & onBF & active1], 'task', 'g', title='AC_onBF_P0A1_gain_per_task')

In [ ]:
plotme(df[AC & onBF & active1], 'task', 'MI', title='AC_onBF_P0A1_MI_per_task')

In [ ]:
plotme(df[AC & offBF & active1], 'task', 'g', title='AC_offBF_P0A1_gain_per_task')

In [ ]:
plotme(df[AC & offBF & active1], 'task', 'd', title='AC_offBF_P0A1_offset_per_task')

In [ ]:
plotme(df[AC & offBF & active1], 'task', 'MI', title='AC_offBF_P0A1_MI_per_task')

In [ ]:
plotme(df[IC & active1], 'animal', 'g', title='IC_P0A1_gain_per_animal')

In [ ]:
plotme(df[IC & passive1], 'animal', 'g', title='IC_P0P1_gain_per_animal')

In [ ]:
plotme(df[IC & active1], 'experimenter', 'g', title='IC_P0A1_gain_per_task')

In [ ]:
plotme(df[IC & active1], 'task', 'MI', title='IC_onBF_P0A1_MI_per_task')

In [ ]:
plotme(df[IC & onBF & active1], 'task', 'g', title='IC_onBF_P0A1_gain_per_task')

In [ ]:
plotme(df[IC & onBF & active1], 'task', 'MI', title='IC_onBF_P0A1_MI_per_task')

In [ ]:
plotme(df[IC & offBF & active1], 'task', 'g', title='IC_offBF_P0A1_gain_per_task')

In [ ]:
plotme(df[IC & offBF & active1], 'task', 'MI', title='IC_offBF_P0A1_MI_per_task')